In [1]:
# %%bash
# cd /core
# git config remote.origin.fetch "+refs/heads/*:refs/remotes/origin/*" && git fetch && git checkout main
# pip install . --no-deps

# cd /dataloader
# git config remote.origin.fetch "+refs/heads/*:refs/remotes/origin/*" && git fetch && git checkout main
# pip install . --no-deps

# cd /nvtabular
# git config remote.origin.fetch "+refs/heads/*:refs/remotes/origin/*" && git fetch && git checkout main
# pip install . --no-deps

# cd /models
# git config remote.origin.fetch "+refs/heads/*:refs/remotes/origin/*" && git fetch && git checkout main
# pip install . --no-deps

# cd /systems
# git config remote.origin.fetch "+refs/heads/*:refs/remotes/origin/*" && git fetch && git checkout main
# pip install . --no-deps

# cd /transformers4rec
# git config remote.origin.fetch "+refs/heads/*:refs/remotes/origin/*" && git fetch && git checkout main
# pip install . --no-deps

In [2]:
# Copyright 2022 NVIDIA Corporation. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions anda
# limitations under the License.
# ==============================================================================

# Each user is responsible for checking the content of datasets and the
# applicable licenses and determining if suitable for the intended use.

<img src="https://developer.download.nvidia.com/notebooks/dlsw-notebooks/merlin_models-transformers-net-item-prediction/nvidia_logo.png" style="width: 90px; float: right;">

# Transformer-based architecture for next-item prediction task with pretrained embeddings

This notebook is created using the latest stable [merlin-tensorflow](https://catalog.ngc.nvidia.com/orgs/nvidia/teams/merlin/containers/merlin-tensorflow/tags) container.

## Overview

In this use case we will train a Transformer-based architecture for next-item prediction task with pretrained embeddings.

**You can chose to download the full dataset manually or use synthetic data.**

We will use the [booking.com dataset](https://github.com/bookingcom/ml-dataset-mdt) to train a session-based model. The dataset contains 1,166,835 of anonymized hotel reservations in the train set and 378,667 in the test set. Each reservation is a part of a customer's trip (identified by `utrip_id`) which includes consecutive reservations.

We will reshape the data to organize it into 'sessions'. Each session will be a full customer itinerary in chronological order. The goal will be to predict the city_id of the final reservation of each trip.


### Learning objectives

- Training a Transformer-based architecture for next-item prediction task

## Downloading and preparing the dataset

You can download the full dataset by registering [here](https://www.coveo.com/en/ailabs/sigir-ecom-data-challenge). If you chose to download the data, please place in alongside this notebook in the `data` directory (you might have to create it).

To process the downloaded data uncomment the cell below.

In [3]:
import nvtabular as nvt
import cudf
import numpy as np
import pandas as pd

2023-06-09 03:56:34.354184: I tensorflow/core/platform/cpu_feature_guard.cc:194] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE3 SSE4.1 SSE4.2 AVX
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
/usr/local/lib/python3.8/dist-packages/merlin/dtypes/mappings/torch.py:43: UserWarning: PyTorch dtype mappings did not load successfully due to an error: No module named 'torch'
  warn(f"PyTorch dtype mappings did not load successfully due to an error: {exc.msg}")
2023-06-09 03:56:35.695417: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-06-09 03:56:35.695768: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:998] successful NUMA node read from SysFS had nega

In [4]:
train = nvt.Dataset('/workspace/sigir_dataset/train/browsing_train.csv', part_size='500MB')
skus = nvt.Dataset('/workspace/sigir_dataset/train/sku_to_content.csv')

The `skus` dataset contains the mapping between the `product_sku_hash` (essentially an item id) to the `description_vector` -- an embedding obtained from the description.

To use this information in our model, we need to map the `product_sku_hash` information to an id.

But we need to make sure that the way we process `skus` and the `train` dataset (event information) is consistent. That the same `product_sku_hash` is mapped to the same id both when processing `skus` and `train`.

We do so by defining and fitting a `Categorify` op and using it to process both datasets.

In [5]:
skus = pd.read_csv('/workspace/sigir_dataset/train/sku_to_content.csv')

skus['description_vector'] = skus['description_vector'].replace(np.nan, '')
skus['image_vector'] = skus['image_vector'].replace(np.nan, '')

skus['description_vector'] = skus['description_vector'].apply(lambda x: [] if len(x) == 0 else eval(x))
skus['image_vector'] = skus['image_vector'].apply(lambda x: [] if len(x) == 0 else eval(x))

In [6]:
cat_op = nvt.ops.Categorify()
out = ['product_sku_hash'] >> cat_op >> nvt.ops.TagAsItemID()
out += ['event_type', 'product_action', 'session_id_hash', 'hashed_url'] >> nvt.ops.Categorify()
out += ['server_timestamp_epoch_ms'] >> nvt.ops.NormalizeMinMax()

wf = nvt.Workflow(out)

train = wf.fit_transform(train)

Now that we have processed the train set, we can use the mapping preserved in the `cat_op` to process the `skus` dataset containing the embeddings we are after.

Let's now `Categorify` the `product_sku_hash` in `skus` and grab just the description embedding information.

In [7]:
skus.head()

,product_sku_hash,description_vector,category_hash,image_vector,price_bucket
0,26ce7b47f4c46e4087e83e54d2f7ddc7ea57862fed2e2a...,[],NaN,[],NaN
1,6383992be772b204a9ab75f86c86f5583d1bdd1222952d...,[],NaN,[],NaN
2,a2c3e2430c6ef9770b903ad08fa067a6b2b9db28f06e1b...,"[0.27629122138023376, -0.15763211250305176, 0....",06fa312761d4b39e2f649781514ac69a4c1505c221fc46...,"[340.3592564184389, -220.19025864725685, 154.0...",7.0
3,1028ef615e425c328e7b95010dfb1fb93cf63749a1bc80...,"[0.4058118760585785, -0.03595402091741562, 0.2...",115a6a7017ee55752b8487c77dfde92b0d501d10a2e69c...,"[180.3463662921092, 222.702322343354, -8.88703...",8.0
4,9870c682d0d52d635501249da0eeaa118fad430b695ea1...,"[-0.3206155300140381, 0.01991105079650879, 0.0...",0665a81d19c89281cc00e7f7d779ded2ed42c933838602...,"[-114.81079301576219, 84.55770104232334, 85.51...",2.0


The data contains `image_vector` information which we won't be using and hence we don't include it in the workflow below.

In [8]:
# import pdb

In [9]:
skus[skus.description_vector.apply(len) > 0].description_vector

2        [0.27629122138023376, -0.15763211250305176, 0....
3        [0.4058118760585785, -0.03595402091741562, 0.2...
4        [-0.3206155300140381, 0.01991105079650879, 0.0...
7        [-0.1854386031627655, 0.19424490630626678, -0....
10       [-0.24601778388023376, -0.12155783176422119, -...
                               ...                        
66375    [-0.2360590547323227, -0.1479017287492752, 0.0...
66376    [-0.1373182237148285, 0.35506564378738403, -0....
66378    [-0.17120949923992157, 0.17244458198547363, -0...
66382    [-0.19150441884994507, -0.06235162168741226, -...
66383    [-0.19760936498641968, 0.4446450471878052, -0....
Name: description_vector, Length: 31950, dtype: object

In [10]:
skus = skus[skus.description_vector.apply(len) > 0]

In [11]:
out = ['product_sku_hash'] >> cat_op
# out += ['product_sku_hash', 'description_vector'] >> nvt.ops.Filter(lambda df: ~df['description_vector'].isna())
# out += ['description_vector'] >> nvt.ops.FillMedian()
wf = nvt.Workflow(out + 'description_vector')
# wf = nvt.Workflow(out)
skus_ds = wf.transform(nvt.Dataset(skus))

In [12]:
skus_ds.head()

,product_sku_hash,description_vector
0,6207,"[0.27629122138023376, -0.15763211250305176, 0...."
1,7691,"[0.4058118760585785, -0.03595402091741562, 0.2..."
2,10812,"[-0.3206155300140381, 0.01991105079650879, 0.0..."
3,21238,"[-0.1854386031627655, 0.19424490630626678, -0...."
4,8776,"[-0.24601778388023376, -0.12155783176422119, -..."


In [13]:
skus_ds.to_npy('skus.npy')

In [29]:
skus_ds.compute()['product_sku_hash']

0         6207
1         7691
2        10812
3        21238
4         8776
         ...  
31945        2
31946     5155
31947    25151
31948        2
31949    28704
Name: product_sku_hash, Length: 31950, dtype: int64

In [26]:
np.load('skus.npy')[:5, 0]

array([ 6207.,  7691., 10812., 21238.,  8776.])

In [30]:
from merlin.dataloader.tensorflow import Loader

In [32]:
embeddings = np.load('skus.npy')

In [35]:
train.compute().shape

(36079307, 6)

In [37]:
train.head()

,product_sku_hash,event_type,product_action,session_id_hash,hashed_url,server_timestamp_epoch_ms
0,247,4,3,66853,376,0.431877
1,100,4,3,66853,197,0.431877
2,1,3,1,66853,197,0.431877
3,247,4,3,66853,376,0.431877
4,1,3,1,66853,376,0.431877


In [39]:
train.compute()['session_id_hash'].nunique()

4934699

In [47]:
train.tail()

,product_sku_hash,event_type,product_action,session_id_hash,hashed_url,server_timestamp_epoch_ms
2116245,1,3,1,257598,5,0.596194
2116246,1,3,1,257598,5,0.596196
2116247,1,3,1,257598,5,0.596197
2116248,1,3,1,257598,3,0.596198
2116249,1,3,1,257598,104,0.596203


In [81]:
train.schema

,name,tags,dtype,is_list,is_ragged,properties.num_buckets,properties.freq_threshold,properties.max_size,properties.cat_path,properties.domain.min,properties.domain.max,properties.domain.name,properties.embedding_sizes.cardinality,properties.embedding_sizes.dimension
0,product_sku_hash,"(Tags.ID, Tags.ITEM, Tags.CATEGORICAL)","DType(name='int64', element_type=<ElementType....",False,False,NaN,0.0,0.0,.//categories/unique.product_sku_hash.parquet,0.0,57485.0,product_sku_hash,57486.0,512.0
1,event_type,(Tags.CATEGORICAL),"DType(name='int64', element_type=<ElementType....",False,False,NaN,0.0,0.0,.//categories/unique.event_type.parquet,0.0,4.0,event_type,5.0,16.0
2,product_action,(Tags.CATEGORICAL),"DType(name='int64', element_type=<ElementType....",False,False,NaN,0.0,0.0,.//categories/unique.product_action.parquet,0.0,6.0,product_action,7.0,16.0
3,session_id_hash,(Tags.CATEGORICAL),"DType(name='int64', element_type=<ElementType....",False,False,NaN,0.0,0.0,.//categories/unique.session_id_hash.parquet,0.0,4934701.0,session_id_hash,4934702.0,512.0
4,hashed_url,(Tags.CATEGORICAL),"DType(name='int64', element_type=<ElementType....",False,False,NaN,0.0,0.0,.//categories/unique.hashed_url.parquet,0.0,489302.0,hashed_url,489303.0,512.0
5,server_timestamp_epoch_ms,(Tags.CONTINUOUS),"DType(name='float64', element_type=<ElementTyp...",False,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [59]:
groupby_features = train.head().columns.tolist() >> nvt.ops.Groupby(
    groupby_cols=['session_id_hash'],
    aggs={
        'product_sku_hash': ['list', 'count'],
        'event_type': ['list'],
        'product_action': ['list'],
        'hashed_url': ['list'],
        'server_timestamp_epoch_ms': ['list']
    },
    sort_cols="server_timestamp_epoch_ms"
)

MINIMUM_SESSION_LENGTH = 5
filtered_sessions = groupby_features >> nvt.ops.Filter(f=lambda df: df["product_sku_hash_count"] >= MINIMUM_SESSION_LENGTH) 

In [63]:
wf = nvt.Workflow(filtered_sessions)
train_processed = wf.fit_transform(train)

In [90]:
df = cudf.DataFrame(data={'id': [0, 0, 1, 2], 'cat': list(range(4)), 'val': [10, 20, 20, 30]})

ds = nvt.Dataset(df)

cat = ['cat'] >> nvt.ops.Categorify()
groupby_features = cat + ['id', 'val'] >> nvt.ops.Groupby(
    groupby_cols=['id'],
    aggs={
        'cat': ['list', 'count'],
        'val': ['list']
    }
)

wf = nvt.Workflow(groupby_features)
ds_transformed = wf.fit_transform(ds)

In [91]:
ds_transformed.schema

,name,tags,dtype,is_list,is_ragged,properties.num_buckets,properties.freq_threshold,properties.max_size,properties.cat_path,properties.domain.min,properties.domain.max,properties.domain.name,properties.embedding_sizes.cardinality,properties.embedding_sizes.dimension,properties.value_count.min,properties.value_count.max
0,id,(),"DType(name='int64', element_type=<ElementType....",False,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,cat_list,(Tags.CATEGORICAL),"DType(name='int64', element_type=<ElementType....",True,True,NaN,0.0,0.0,.//categories/unique.cat.parquet,0.0,6.0,cat,7.0,16.0,0.0,NaN
2,val_list,(),"DType(name='int64', element_type=<ElementType....",True,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN
3,cat_count,(Tags.CATEGORICAL),"DType(name='int32', element_type=<ElementType....",False,False,NaN,0.0,0.0,.//categories/unique.cat.parquet,0.0,6.0,cat,7.0,16.0,NaN,NaN


In [93]:
ds_transformed.compute()

,id,cat_list,val_list,cat_count
0,0,"[3, 4]","[10, 20]",2
1,1,[5],[20],1
2,2,[6],[30],1


In [82]:
train_processed.schema

,name,tags,dtype,is_list,is_ragged,properties.num_buckets,properties.freq_threshold,properties.max_size,properties.cat_path,properties.domain.min,properties.domain.max,properties.domain.name,properties.embedding_sizes.cardinality,properties.embedding_sizes.dimension,properties.value_count.min,properties.value_count.max
0,session_id_hash,(Tags.CATEGORICAL),"DType(name='int64', element_type=<ElementType....",False,False,NaN,0.0,0.0,.//categories/unique.session_id_hash.parquet,0.0,4934701.0,session_id_hash,4934702.0,512.0,NaN,NaN
1,product_sku_hash_list,"(Tags.ID, Tags.ITEM, Tags.CATEGORICAL)","DType(name='int64', element_type=<ElementType....",True,True,NaN,0.0,0.0,.//categories/unique.product_sku_hash.parquet,0.0,57485.0,product_sku_hash,57486.0,512.0,0.0,NaN
2,event_type_list,(Tags.CATEGORICAL),"DType(name='int64', element_type=<ElementType....",True,True,NaN,0.0,0.0,.//categories/unique.event_type.parquet,0.0,4.0,event_type,5.0,16.0,0.0,NaN
3,product_action_list,(Tags.CATEGORICAL),"DType(name='int64', element_type=<ElementType....",True,True,NaN,0.0,0.0,.//categories/unique.product_action.parquet,0.0,6.0,product_action,7.0,16.0,0.0,NaN
4,hashed_url_list,(Tags.CATEGORICAL),"DType(name='int64', element_type=<ElementType....",True,True,NaN,0.0,0.0,.//categories/unique.hashed_url.parquet,0.0,489302.0,hashed_url,489303.0,512.0,0.0,NaN
5,server_timestamp_epoch_ms_list,(Tags.CONTINUOUS),"DType(name='float64', element_type=<ElementTyp...",True,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN
6,product_sku_hash_count,"(Tags.ID, Tags.ITEM, Tags.CATEGORICAL)","DType(name='int32', element_type=<ElementType....",False,False,NaN,0.0,0.0,.//categories/unique.product_sku_hash.parquet,0.0,57485.0,product_sku_hash,57486.0,512.0,NaN,NaN


In [64]:
train_processed.head()

,session_id_hash,product_sku_hash_list,event_type_list,product_action_list,hashed_url_list,server_timestamp_epoch_ms_list,product_sku_hash_count
0,19,"[1, 3, 1, 17792, 3, 3, 1, 3, 3, 3, 1, 1, 1, 1,...","[3, 4, 3, 4, 4, 4, 3, 4, 4, 4, 3, 3, 3, 3, 3, ...","[1, 5, 1, 5, 5, 5, 1, 5, 5, 5, 1, 1, 1, 1, 1, ...","[3, 5, 5, 183199, 183199, 5, 5, 157277, 157277...","[0.017701422675646814, 0.017701613740402588, 0...",200
1,28,"[49, 1, 1, 1, 1, 2779, 1, 1, 1, 1, 1, 1, 1, 1,...","[4, 3, 3, 3, 3, 4, 3, 3, 3, 3, 3, 3, 3, 3, 3, ...","[3, 1, 1, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[126, 126, 19, 11, 4241, 3689, 3689, 4241, 695...","[0.08774476140013235, 0.08774476140013235, 0.0...",200
2,31,"[1, 1, 1, 1, 381, 1, 1, 1, 1, 1, 1, 1, 1, 1086...","[3, 3, 3, 3, 4, 3, 3, 3, 3, 3, 3, 3, 3, 4, 3, ...","[1, 1, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 1, 3, 1, ...","[3, 77, 3802, 663, 663, 3802, 77, 77, 95, 77, ...","[0.41732644544485625, 0.4173890392881526, 0.41...",200
3,47,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 297, 1, 413, 1,...","[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 3, 4, 3, 4, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 1, 3, 1, 3, ...","[5648, 2648, 2648, 13, 53, 14323, 22235, 12033...","[0.7742340701027528, 0.7742346497301769, 0.774...",200
4,73,"[1, 1, 3287, 1, 1, 2349, 3287, 1, 1, 2349, 328...","[3, 3, 4, 3, 3, 4, 4, 3, 3, 4, 4, 3, 3, 4, 3, ...","[1, 1, 3, 1, 1, 3, 3, 1, 1, 3, 3, 1, 1, 3, 1, ...","[40, 13, 4411, 4411, 3075, 3075, 4411, 4411, 3...","[0.3928006023156755, 0.3928008996561809, 0.392...",200


In [65]:
train_processed.compute().shape

(1884102, 7)

In [ ]:
nvt.Workflow()

In [66]:
from merlin.dataloader.ops.embeddings import EmbeddingOperator

In [188]:
embeddings

array([[ 6.20700000e+03,  2.76291221e-01, -1.57632113e-01, ...,
         1.37491841e-02, -6.54154569e-02,  2.93405820e-02],
       [ 7.69100000e+03,  4.05811876e-01, -3.59540209e-02, ...,
        -5.11377156e-02,  1.23936273e-02, -3.28034014e-02],
       [ 1.08120000e+04, -3.20615530e-01,  1.99110508e-02, ...,
         4.23921552e-03,  1.48407519e-02, -8.13788921e-03],
       ...,
       [ 2.51510000e+04, -1.71209499e-01,  1.72444582e-01, ...,
         3.26483995e-02, -1.27059463e-02, -2.08097659e-02],
       [ 2.00000000e+00, -1.91504419e-01, -6.23516217e-02, ...,
        -2.50775218e-02, -2.31876411e-02,  4.92795147e-02],
       [ 2.87040000e+04, -1.97609365e-01,  4.44645047e-01, ...,
         4.00494635e-02,  3.62532213e-03,  4.55308110e-02]])

In [189]:
loader = Loader(
    train_processed,
    batch_size=1,
    transforms=[
        EmbeddingOperator(
            embeddings[:, 1:],
            id_lookup_table=embeddings[:, 0].astype(int),
            lookup_key="product_sku_hash_list",
        )
    ],
    shuffle=True
)

/usr/local/lib/python3.8/dist-packages/merlin/dataloader/tensorflow.py:65: UserWarning: Due to a CUDA memory alignment issue in some Tensorflow operations such as Embedding ops, we recommend that 'batch_size' be at least 16 and also a power of two. Please change 'batch_size' to a number that is a power of two that is greater than or equal to 16.
  warnings.warn(


In [202]:
np.where(embeddings[:, 0] == 1221)

(array([15456]),)

In [204]:
embeddings[15456, 1:]

array([ 0.44937021, -0.03551548,  0.31595686, -0.03524658, -0.07513202,
       -0.01411778, -0.00121029,  0.04885858, -0.08007973, -0.12252348,
        0.0007462 , -0.02797323, -0.07026914, -0.0429178 , -0.03331489,
        0.01377482, -0.03244033, -0.02438913,  0.06625758, -0.03113006,
       -0.11172944,  0.0808474 , -0.03310855,  0.01004405, -0.08447912,
        0.01327929,  0.00186707, -0.03876101,  0.06090691,  0.14134969,
       -0.02354587,  0.01042901, -0.19002861, -0.06321244, -0.06413285,
        0.05125846,  0.07666769,  0.08735777,  0.06115319, -0.08536364,
       -0.03397714, -0.01086652, -0.07932654,  0.0039208 , -0.06790888,
        0.03998983, -0.02066554,  0.00341779,  0.01210334, -0.04248429])

In [190]:
loader.peek()

({'session_id_hash': <tf.Tensor: shape=(1,), dtype=int64, numpy=array([841218])>,
  'product_sku_hash_list__values': <tf.Tensor: shape=(11,), dtype=int64, numpy=array([   1,    1,    1,    1,    1, 1221,    1,    1,    1,    1,    1])>,
  'product_sku_hash_list__offsets': <tf.Tensor: shape=(2,), dtype=int32, numpy=array([ 0, 11], dtype=int32)>,
  'event_type_list__values': <tf.Tensor: shape=(11,), dtype=int64, numpy=array([3, 3, 3, 3, 3, 4, 3, 3, 3, 3, 3])>,
  'event_type_list__offsets': <tf.Tensor: shape=(2,), dtype=int32, numpy=array([ 0, 11], dtype=int32)>,
  'product_action_list__values': <tf.Tensor: shape=(11,), dtype=int64, numpy=array([1, 1, 1, 1, 1, 3, 1, 1, 1, 1, 1])>,
  'product_action_list__offsets': <tf.Tensor: shape=(2,), dtype=int32, numpy=array([ 0, 11], dtype=int32)>,
  'hashed_url_list__values': <tf.Tensor: shape=(11,), dtype=int64, numpy=array([   3,   90, 1403,   90,  628, 1667, 1667,  628,   11,   25,   11])>,
  'hashed_url_list__offsets': <tf.Tensor: shape=(2,), dt

In [73]:
import merlin.models.tf as mm

[INFO]: sparse_operation_kit is imported
[SOK INFO] Import /usr/local/lib/python3.8/dist-packages/merlin_sok-1.1.4-py3.8-linux-x86_64.egg/sparse_operation_kit/lib/libsok_experiment.so
[SOK INFO] Import /usr/local/lib/python3.8/dist-packages/merlin_sok-1.1.4-py3.8-linux-x86_64.egg/sparse_operation_kit/lib/libsok_experiment.so
[SOK INFO] Initialize finished, communication tool: horovod


In [76]:
from merlin.schema import ColumnSchema, Schema, Tags

In [79]:
train_processed.schema.select_by_tag(Tags.CATEGORICAL)

,name,tags,dtype,is_list,is_ragged,properties.num_buckets,properties.freq_threshold,properties.max_size,properties.cat_path,properties.domain.min,properties.domain.max,properties.domain.name,properties.embedding_sizes.cardinality,properties.embedding_sizes.dimension,properties.value_count.min,properties.value_count.max
0,session_id_hash,(Tags.CATEGORICAL),"DType(name='int64', element_type=<ElementType....",False,False,None,0,0,.//categories/unique.session_id_hash.parquet,0,4934701,session_id_hash,4934702,512,NaN,NaN
1,product_sku_hash_list,"(Tags.ID, Tags.ITEM, Tags.CATEGORICAL)","DType(name='int64', element_type=<ElementType....",True,True,None,0,0,.//categories/unique.product_sku_hash.parquet,0,57485,product_sku_hash,57486,512,0.0,NaN
2,event_type_list,(Tags.CATEGORICAL),"DType(name='int64', element_type=<ElementType....",True,True,None,0,0,.//categories/unique.event_type.parquet,0,4,event_type,5,16,0.0,NaN
3,product_action_list,(Tags.CATEGORICAL),"DType(name='int64', element_type=<ElementType....",True,True,None,0,0,.//categories/unique.product_action.parquet,0,6,product_action,7,16,0.0,NaN
4,hashed_url_list,(Tags.CATEGORICAL),"DType(name='int64', element_type=<ElementType....",True,True,None,0,0,.//categories/unique.hashed_url.parquet,0,489302,hashed_url,489303,512,0.0,NaN
5,product_sku_hash_count,"(Tags.ID, Tags.ITEM, Tags.CATEGORICAL)","DType(name='int32', element_type=<ElementType....",False,False,None,0,0,.//categories/unique.product_sku_hash.parquet,0,57485,product_sku_hash,57486,512,NaN,NaN


In [80]:
train_processed.schema

,name,tags,dtype,is_list,is_ragged,properties.num_buckets,properties.freq_threshold,properties.max_size,properties.cat_path,properties.domain.min,properties.domain.max,properties.domain.name,properties.embedding_sizes.cardinality,properties.embedding_sizes.dimension,properties.value_count.min,properties.value_count.max
0,session_id_hash,(Tags.CATEGORICAL),"DType(name='int64', element_type=<ElementType....",False,False,NaN,0.0,0.0,.//categories/unique.session_id_hash.parquet,0.0,4934701.0,session_id_hash,4934702.0,512.0,NaN,NaN
1,product_sku_hash_list,"(Tags.ID, Tags.ITEM, Tags.CATEGORICAL)","DType(name='int64', element_type=<ElementType....",True,True,NaN,0.0,0.0,.//categories/unique.product_sku_hash.parquet,0.0,57485.0,product_sku_hash,57486.0,512.0,0.0,NaN
2,event_type_list,(Tags.CATEGORICAL),"DType(name='int64', element_type=<ElementType....",True,True,NaN,0.0,0.0,.//categories/unique.event_type.parquet,0.0,4.0,event_type,5.0,16.0,0.0,NaN
3,product_action_list,(Tags.CATEGORICAL),"DType(name='int64', element_type=<ElementType....",True,True,NaN,0.0,0.0,.//categories/unique.product_action.parquet,0.0,6.0,product_action,7.0,16.0,0.0,NaN
4,hashed_url_list,(Tags.CATEGORICAL),"DType(name='int64', element_type=<ElementType....",True,True,NaN,0.0,0.0,.//categories/unique.hashed_url.parquet,0.0,489302.0,hashed_url,489303.0,512.0,0.0,NaN
5,server_timestamp_epoch_ms_list,(Tags.CONTINUOUS),"DType(name='float64', element_type=<ElementTyp...",True,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN
6,product_sku_hash_count,"(Tags.ID, Tags.ITEM, Tags.CATEGORICAL)","DType(name='int32', element_type=<ElementType....",False,False,NaN,0.0,0.0,.//categories/unique.product_sku_hash.parquet,0.0,57485.0,product_sku_hash,57486.0,512.0,NaN,NaN


In [100]:
train_processed.schema.remove_col('product_sku_hash_count').select_by_tag(Tags.CATEGORICAL)

,name,tags,dtype,is_list,is_ragged,properties.num_buckets,properties.freq_threshold,properties.max_size,properties.cat_path,properties.domain.min,properties.domain.max,properties.domain.name,properties.embedding_sizes.cardinality,properties.embedding_sizes.dimension,properties.value_count.min,properties.value_count.max
0,session_id_hash,(Tags.CATEGORICAL),"DType(name='int64', element_type=<ElementType....",False,False,None,0,0,.//categories/unique.session_id_hash.parquet,0,4934701,session_id_hash,4934702,512,NaN,NaN
1,product_sku_hash_list,"(Tags.ID, Tags.ITEM, Tags.CATEGORICAL)","DType(name='int64', element_type=<ElementType....",True,True,None,0,0,.//categories/unique.product_sku_hash.parquet,0,57485,product_sku_hash,57486,512,0.0,NaN
2,event_type_list,(Tags.CATEGORICAL),"DType(name='int64', element_type=<ElementType....",True,True,None,0,0,.//categories/unique.event_type.parquet,0,4,event_type,5,16,0.0,NaN
3,product_action_list,(Tags.CATEGORICAL),"DType(name='int64', element_type=<ElementType....",True,True,None,0,0,.//categories/unique.product_action.parquet,0,6,product_action,7,16,0.0,NaN
4,hashed_url_list,(Tags.CATEGORICAL),"DType(name='int64', element_type=<ElementType....",True,True,None,0,0,.//categories/unique.hashed_url.parquet,0,489302,hashed_url,489303,512,0.0,NaN


In [98]:
train_processed.schema

,name,tags,dtype,is_list,is_ragged,properties.num_buckets,properties.freq_threshold,properties.max_size,properties.cat_path,properties.domain.min,properties.domain.max,properties.domain.name,properties.embedding_sizes.cardinality,properties.embedding_sizes.dimension,properties.value_count.min,properties.value_count.max
0,session_id_hash,(Tags.CATEGORICAL),"DType(name='int64', element_type=<ElementType....",False,False,NaN,0.0,0.0,.//categories/unique.session_id_hash.parquet,0.0,4934701.0,session_id_hash,4934702.0,512.0,NaN,NaN
1,product_sku_hash_list,"(Tags.ID, Tags.ITEM, Tags.CATEGORICAL)","DType(name='int64', element_type=<ElementType....",True,True,NaN,0.0,0.0,.//categories/unique.product_sku_hash.parquet,0.0,57485.0,product_sku_hash,57486.0,512.0,0.0,NaN
2,event_type_list,(Tags.CATEGORICAL),"DType(name='int64', element_type=<ElementType....",True,True,NaN,0.0,0.0,.//categories/unique.event_type.parquet,0.0,4.0,event_type,5.0,16.0,0.0,NaN
3,product_action_list,(Tags.CATEGORICAL),"DType(name='int64', element_type=<ElementType....",True,True,NaN,0.0,0.0,.//categories/unique.product_action.parquet,0.0,6.0,product_action,7.0,16.0,0.0,NaN
4,hashed_url_list,(Tags.CATEGORICAL),"DType(name='int64', element_type=<ElementType....",True,True,NaN,0.0,0.0,.//categories/unique.hashed_url.parquet,0.0,489302.0,hashed_url,489303.0,512.0,0.0,NaN
5,server_timestamp_epoch_ms_list,(Tags.CONTINUOUS),"DType(name='float64', element_type=<ElementTyp...",True,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN
6,product_sku_hash_count,"(Tags.ID, Tags.ITEM, Tags.CATEGORICAL)","DType(name='int32', element_type=<ElementType....",False,False,NaN,0.0,0.0,.//categories/unique.product_sku_hash.parquet,0.0,57485.0,product_sku_hash,57486.0,512.0,NaN,NaN


In [99]:
train_processed.schema.select_by_name('product_sku_hash_count')

,name,tags,dtype,is_list,is_ragged,properties.num_buckets,properties.freq_threshold,properties.max_size,properties.cat_path,properties.domain.min,properties.domain.max,properties.domain.name,properties.embedding_sizes.cardinality,properties.embedding_sizes.dimension
0,product_sku_hash_count,"(Tags.ID, Tags.ITEM, Tags.CATEGORICAL)","DType(name='int32', element_type=<ElementType....",False,False,None,0,0,.//categories/unique.product_sku_hash.parquet,0,57485,product_sku_hash,57486,512


In [215]:
train_processed.schema

,name,tags,dtype,is_list,is_ragged,properties.num_buckets,properties.freq_threshold,properties.max_size,properties.cat_path,properties.domain.min,properties.domain.max,properties.domain.name,properties.embedding_sizes.cardinality,properties.embedding_sizes.dimension,properties.value_count.min,properties.value_count.max
0,session_id_hash,(Tags.CATEGORICAL),"DType(name='int64', element_type=<ElementType....",False,False,NaN,0.0,0.0,.//categories/unique.session_id_hash.parquet,0.0,4934701.0,session_id_hash,4934702.0,512.0,NaN,NaN
1,product_sku_hash_list,"(Tags.ID, Tags.ITEM, Tags.CATEGORICAL)","DType(name='int64', element_type=<ElementType....",True,True,NaN,0.0,0.0,.//categories/unique.product_sku_hash.parquet,0.0,57485.0,product_sku_hash,57486.0,512.0,0.0,NaN
2,event_type_list,(Tags.CATEGORICAL),"DType(name='int64', element_type=<ElementType....",True,True,NaN,0.0,0.0,.//categories/unique.event_type.parquet,0.0,4.0,event_type,5.0,16.0,0.0,NaN
3,product_action_list,(Tags.CATEGORICAL),"DType(name='int64', element_type=<ElementType....",True,True,NaN,0.0,0.0,.//categories/unique.product_action.parquet,0.0,6.0,product_action,7.0,16.0,0.0,NaN
4,hashed_url_list,(Tags.CATEGORICAL),"DType(name='int64', element_type=<ElementType....",True,True,NaN,0.0,0.0,.//categories/unique.hashed_url.parquet,0.0,489302.0,hashed_url,489303.0,512.0,0.0,NaN
5,server_timestamp_epoch_ms_list,(Tags.CONTINUOUS),"DType(name='float64', element_type=<ElementTyp...",True,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN
6,product_sku_hash_count,"(Tags.ID, Tags.ITEM, Tags.CATEGORICAL)","DType(name='int32', element_type=<ElementType....",False,False,NaN,0.0,0.0,.//categories/unique.product_sku_hash.parquet,0.0,57485.0,product_sku_hash,57486.0,512.0,NaN,NaN


In [217]:
schema = loader.output_schema

In [218]:
schema

,name,tags,dtype,is_list,is_ragged,properties.num_buckets,properties.freq_threshold,properties.max_size,properties.cat_path,properties.domain.min,properties.domain.max,properties.domain.name,properties.embedding_sizes.cardinality,properties.embedding_sizes.dimension,properties.value_count.min,properties.value_count.max
0,session_id_hash,(Tags.CATEGORICAL),"DType(name='int64', element_type=<ElementType....",False,False,NaN,0.0,0.0,.//categories/unique.session_id_hash.parquet,0.0,4934701.0,session_id_hash,4934702.0,512.0,NaN,NaN
1,product_sku_hash_list,"(Tags.ID, Tags.ITEM, Tags.CATEGORICAL)","DType(name='int64', element_type=<ElementType....",True,True,NaN,0.0,0.0,.//categories/unique.product_sku_hash.parquet,0.0,57485.0,product_sku_hash,57486.0,512.0,0.0,NaN
2,event_type_list,(Tags.CATEGORICAL),"DType(name='int64', element_type=<ElementType....",True,True,NaN,0.0,0.0,.//categories/unique.event_type.parquet,0.0,4.0,event_type,5.0,16.0,0.0,NaN
3,product_action_list,(Tags.CATEGORICAL),"DType(name='int64', element_type=<ElementType....",True,True,NaN,0.0,0.0,.//categories/unique.product_action.parquet,0.0,6.0,product_action,7.0,16.0,0.0,NaN
4,hashed_url_list,(Tags.CATEGORICAL),"DType(name='int64', element_type=<ElementType....",True,True,NaN,0.0,0.0,.//categories/unique.hashed_url.parquet,0.0,489302.0,hashed_url,489303.0,512.0,0.0,NaN
5,server_timestamp_epoch_ms_list,(Tags.CONTINUOUS),"DType(name='float64', element_type=<ElementTyp...",True,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN
6,product_sku_hash_count,"(Tags.ID, Tags.ITEM, Tags.CATEGORICAL)","DType(name='int32', element_type=<ElementType....",False,False,NaN,0.0,0.0,.//categories/unique.product_sku_hash.parquet,0.0,57485.0,product_sku_hash,57486.0,512.0,NaN,NaN
7,embeddings,"(Tags.EMBEDDING, Tags.ITEM)","DType(name='float64', element_type=<ElementTyp...",True,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN


In [219]:
# if broadcast_non_seq_features:
#     # TODO: Check if it would be possible for EmbeddingOperator to keep the Tags.SEQUENCE
#     # in the output embedding schema if lookup_key has that tag
#     seq_schema = schema.select_by_tag(Tags.SEQUENCE) + schema.select_by_name(
#         "pretrained_item_id_embeddings"
#     )
#     non_seq_schema = schema.select_by_name(["user_country", "pretrained_user_id_embeddings"])
#     input_kwargs = {"post": mm.BroadcastToSequence(non_seq_schema, seq_schema)}

input_block = mm.InputBlockV2(
    schema,
    embeddings=mm.Embeddings(
        schema.remove_col('product_sku_hash_count').select_by_tag(Tags.CATEGORICAL),
        sequence_combiner=None,
    ),
    pretrained_embeddings=mm.PretrainedEmbeddings(
        schema.select_by_tag(Tags.EMBEDDING),
        sequence_combiner=None,
        normalizer="l2-norm",
        output_dims={"embeddings": 128},
    ),
    aggregation=None,
#     post=mm.BroadcastToSequence(non_seq_schema, seq_schema)
)

In [220]:
input_block

ParallelBlock(
  (parallel_layers): Dict(
    (categorical): ParallelBlock(
      (parallel_layers): Dict(
        (session_id_hash): EmbeddingTable(
          (features): Dict(
            (session_id_hash): ColumnSchema(name='session_id_hash', tags={<Tags.CATEGORICAL: 'categorical'>}, properties={'num_buckets': None, 'freq_threshold': 0, 'max_size': 0, 'cat_path': './/categories/unique.session_id_hash.parquet', 'domain': {'min': 0, 'max': 4934701, 'name': 'session_id_hash'}, 'embedding_sizes': {'cardinality': 4934702, 'dimension': 512}}, dtype=DType(name='int64', element_type=<ElementType.Int: 'int'>, element_size=64, element_unit=None, signed=True, shape=Shape(dims=(Dimension(min=0, max=None),))), is_list=False, is_ragged=False)
          )
          (table): Embedding()
        )
        (product_sku_hash): EmbeddingTable(
          (features): Dict(
            (product_sku_hash_list): ColumnSchema(name='product_sku_hash_list', tags={<Tags.ID: 'id'>, <Tags.ITEM: 'item'>, <Tags.CAT

In [221]:
b = loader.peek()

In [222]:
b

({'session_id_hash': <tf.Tensor: shape=(1,), dtype=int64, numpy=array([841218])>,
  'product_sku_hash_list__values': <tf.Tensor: shape=(11,), dtype=int64, numpy=array([   1,    1,    1,    1,    1, 1221,    1,    1,    1,    1,    1])>,
  'product_sku_hash_list__offsets': <tf.Tensor: shape=(2,), dtype=int32, numpy=array([ 0, 11], dtype=int32)>,
  'event_type_list__values': <tf.Tensor: shape=(11,), dtype=int64, numpy=array([3, 3, 3, 3, 3, 4, 3, 3, 3, 3, 3])>,
  'event_type_list__offsets': <tf.Tensor: shape=(2,), dtype=int32, numpy=array([ 0, 11], dtype=int32)>,
  'product_action_list__values': <tf.Tensor: shape=(11,), dtype=int64, numpy=array([1, 1, 1, 1, 1, 3, 1, 1, 1, 1, 1])>,
  'product_action_list__offsets': <tf.Tensor: shape=(2,), dtype=int32, numpy=array([ 0, 11], dtype=int32)>,
  'hashed_url_list__values': <tf.Tensor: shape=(11,), dtype=int64, numpy=array([   3,   90, 1403,   90,  628, 1667, 1667,  628,   11,   25,   11])>,
  'hashed_url_list__offsets': <tf.Tensor: shape=(2,), dt

In [223]:
inputs = mm.sample_batch(loader, batch_size=10, include_targets=False, prepare_features=True)
input_batch = input_block(inputs)

In [224]:
input_batch.keys()

dict_keys(['session_id_hash', 'product_sku_hash_list', 'event_type_list', 'product_action_list', 'hashed_url_list', 'server_timestamp_epoch_ms_list', 'embeddings'])

In [228]:
input_batch['embeddings'][0][5].shape

TensorShape([128])

In [ ]:
    embeddings = np.load(npy_path)
    # second workflow that categorifies the embedding table data
    df = make_df({"string_id": np.random.choice(string_ids, 30)})
    graph2 = ["string_id"] >> cat_op
    train_res = Workflow(graph2).transform(Dataset(df, cpu=(cpu is not None)))

    data_loader = Loader(
        train_res,
        batch_size=1,
        transforms=[
            EmbeddingOperator(
                embeddings[:, 1:],
                id_lookup_table=embeddings[:, 0].astype(int),
                lookup_key="string_id",
            )
        ],
        shuffle=False,
        device=cpu,
    )
    origin_df = train_res.to_ddf().merge(emb_res.to_ddf(), on="string_id", how="left").compute()
    for idx, batch in enumerate(data_loader):
        batch
        b_df = batch[0].to_df()
        org_df = origin_df.iloc[idx]
        if not cpu:
            assert (b_df["string_id"].to_numpy() == org_df["string_id"].to_numpy()).all()
            assert (b_df["embeddings"].list.leaves == org_df["embeddings"].list.leaves).all()
        else:
            assert (b_df["string_id"].values == org_df["string_id"]).all()
            assert b_df["embeddings"].values[0] == org_df["embeddings"].tolist()

In [ ]:
import cudf
import numpy as np

In [ ]:
df = cudf.DataFrame(data={'id': [1,2,3], 'val': [0, np.nan, 10], 'another_col': ['a', 'b', 'c']})

In [ ]:
df.val[df.val.isna()]

In [ ]:
df.val[~df.val.isna()]

In [ ]:
ds = nvt.Dataset(df)

out = ['val'] >> nvt.ops.Filter(f=lambda col: ~col.isna())

wf = nvt.Workflow(out)
wf.fit_transform(ds).compute()

In [ ]:
out = ['id', 'val'] >> nvt.ops.Filter(f=lambda df: ~df['val'].isna())

wf = nvt.Workflow(out)
wf.fit_transform(ds).compute()

In [ ]:
out = ['id', 'val'] >> nvt.ops.Filter(f=lambda df: ~df['val'].isna())

wf = nvt.Workflow(out + ['another_col'])
wf.fit_transform(ds).compute()

In [ ]:
train.head()

In [ ]:
skus.to_npy('embeddings.npy')

In [ ]:
out = ['product_sku_hash', 'category_hash'] >> nvt.ops.Categorify() >> nvt.ops.TagAsItemID()
out += ['description_vector'] >> nvt.ops.TagAsItemFeatures()
out += ['price_bucket'] >> nvt.ops.NormalizeMinMax()

wf = nvt.Workflow(out)
skus = wf.fit_transform(skus)

In [ ]:
train.head()

In [ ]:
skus.head()

To use synthetically generated data, uncomment the following cell:

In [ ]:
%%bash

cd /workspace && pip install . 

In [ ]:
from merlin.datasets.synthetic import KNOWN_DATASETS

In [ ]:
KNOWN_DATASETS

In [ ]:
from merlin.datasets.synthetic import generate_data

generate_data('sigir-browsing', 1000).head()

In [ ]:
generate_data('sigir-sku', 1000).head()